In [1]:
%load_ext autoreload
%autoreload 2
import os
import sys
import numpy as np
import pandas as pd
import pingouin as pg
import plotly.graph_objects as go
from os.path import join as pjoin
from natsort import natsorted

sys.path.append("../../")
import circletrack_behavior as ctb
import plotting_functions as pf

/home/austinbaggetta/.conda/envs/calcium/lib/python3.10/site-packages/outdated/utils.py:14: OutdatedPackageWarning: The package pingouin is out of date. Your version is 0.5.2, the latest is 0.5.3.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(


### The three blocks below can be used during an experiment to track changes in performance in real time.

In [3]:
## Set path variables
mouse_list = ['mc27', 'mc28']
behavior_path = '../../../MultiCon_Imaging/MultiCon_Imaging3/output/behav/'
subject_color = 'darkgrey'
chance_color = 'darkgrey'
mouse_colors = {'mc27': 'turquoise', 'mc28': 'darkorchid'}
session_list = [f'A{x}' for x in np.arange(1, 6)] + [f'B{x}' for x in np.arange(1, 6)] + [f'C{x}' for x in np.arange(1, 6)] + [f'D{x}' for x in np.arange(1, 6)]
bin_size = 5

In [6]:
## Create file list dataframe
behavior_type = 'circletrack_data'
csv_name = 'circle_track.csv'
dpath = os.path.abspath('../../../MultiCon_Imaging/MultiCon_Imaging3/{}/data/**/**/**/{}'.format(behavior_type, csv_name))
file_list = ctb.get_file_list(dpath)
mouse_id = []
for file in file_list:
    mouse_id.append(ctb.get_mouse(file, str2match='(mc[0-9]+)'))
combined = ctb.combine(file_list, mouse_id)

In [7]:
lick_df = pd.DataFrame()
for mouse in np.unique(mouse_id):
    lick_data = ctb.get_lick_accuracy(combined, mouse)
    lick_df = pd.concat([lick_df, lick_data], ignore_index=True)
## Lick accuracy plot
fig = pf.plot_behavior_across_days(lick_df, x_var='day', y_var='percent_correct', groupby_var=['day'], plot_transitions=[5.5, 10.5, 15.5],
                                   marker_color=subject_color, avg_color='turquoise', transition_color=['darkgrey', 'darkgrey', 'darkgrey'],
                                   x_title='Day', y_title='Lick Accuracy (%)', titles=['Circle Track: Lick Accuracy'], height=500, width=500)
fig.show()

### The code below looks at behavior that has been preprocessed.

In [8]:
## Results from processed behavior data to look at probe performance
behav_dict = {'mouse': [], 'day': [], 'session': [], 'pc_nonprobe': [], 'pc_probe': []}
for mouse in mouse_list:
    mouse_path = pjoin(behavior_path, mouse)
    for idx, session in enumerate(os.listdir(mouse_path)):
        behav = pd.read_feather(pjoin(mouse_path, f'{session}'))
        behav_nonprobe = behav[~behav['probe']]
        behav_probe = behav[behav['probe']]
        reward_one, reward_two = np.unique(behav['reward_one'])[0], np.unique(behav['reward_two'])[0]
        ## Percent correct for non-probe behavior
        pc = ctb.lick_accuracy(behav_nonprobe, port_one=reward_one, port_two=reward_two, by_trials=False)
        pc_probe = ctb.lick_accuracy(behav_probe, port_one=reward_one, port_two=reward_two, by_trials=False)
        behav_dict['mouse'].append(mouse)
        behav_dict['day'].append(idx + 1)
        behav_dict['session'].append(np.unique(behav['session'])[0])
        behav_dict['pc_nonprobe'].append(pc) 
        behav_dict['pc_probe'].append(pc_probe)
behav_df = pd.DataFrame(behav_dict)

/media/caishuman/csstorage3/Austin/CircleTrack/CircleTrackAnalysis/MultiCon_Imaging/MultiCon_Imaging3/../../circletrack_behavior.py:1067: RuntimeWarning:

invalid value encountered in scalar divide

/media/caishuman/csstorage3/Austin/CircleTrack/CircleTrackAnalysis/MultiCon_Imaging/MultiCon_Imaging3/../../circletrack_behavior.py:1067: RuntimeWarning:

invalid value encountered in scalar divide

/media/caishuman/csstorage3/Austin/CircleTrack/CircleTrackAnalysis/MultiCon_Imaging/MultiCon_Imaging3/../../circletrack_behavior.py:1067: RuntimeWarning:

invalid value encountered in scalar divide

/media/caishuman/csstorage3/Austin/CircleTrack/CircleTrackAnalysis/MultiCon_Imaging/MultiCon_Imaging3/../../circletrack_behavior.py:1067: RuntimeWarning:

invalid value encountered in scalar divide

/media/caishuman/csstorage3/Austin/CircleTrack/CircleTrackAnalysis/MultiCon_Imaging/MultiCon_Imaging3/../../circletrack_behavior.py:1067: RuntimeWarning:

invalid value encountered in scalar divide

/medi

In [9]:
fig = pf.plot_behavior_across_days(behav_df, x_var='day', y_var='pc_nonprobe', groupby_var=['day'], plot_transitions=[5.5, 10.5, 15.5],
                                   marker_color=subject_color, avg_color='turquoise', transition_color=['darkgrey', 'darkgrey', 'darkgrey'],
                                   x_title='Day', y_title='Lick Accuracy (%)', titles=['Circle Track: Lick Accuracy'], height=500, width=500)
fig.show()

In [10]:
## Plot first and last day percent correct during probe across different contexts
first_last = pd.DataFrame()
context_list = ['A', 'B', 'C']
for mouse in mouse_list:
    mouse_data = behav_df[behav_df['mouse'] == mouse]
    index_list = ctb.pick_context_day(mouse_data, day_index=0, contexts=context_list)
    index_list_two = ctb.pick_context_day(mouse_data, day_index=-1, contexts=context_list)
    sub_data = mouse_data.loc[index_list, :]
    sub_data.insert(0, 'day_type', 'First')
    sub_data_two = mouse_data.loc[index_list_two, :]
    sub_data_two.insert(0, 'day_type', 'Last')
    first_last = pd.concat([first_last, sub_data, sub_data_two])
avg_combined = first_last.groupby(['day_type', 'session'], as_index=False).agg({'pc_probe': ['mean', 'sem']})

## Plot 
fig = pf.custom_graph_template(x_title='', y_title='', width=800, rows=1, columns=len(context_list), 
                               titles=['A', 'B', 'C', 'D'], shared_x=True, shared_y=True)
for idx, session in enumerate(context_list):
    plot_data = avg_combined[avg_combined['session'] == session]
    fig.add_trace(go.Scatter(x=plot_data['day_type'], y=plot_data['pc_probe']['mean'],
                                error_y=dict(type='data', array=plot_data['pc_probe']['sem'], thickness=1.5, width=8), 
                                line_color='turquoise', showlegend=False), row=1, col=idx+1)
    fig.add_hline(y=25, line_width=1, line_dash='dash', line_color='darkgrey', opacity=1)

for mouse in np.unique(first_last['mouse']):
    mouse_data = first_last[first_last['mouse'] == mouse]
    for idx, session in enumerate(np.unique(mouse_data['session'])):
        session_data = mouse_data[mouse_data['session'] == session]
        fig.add_trace(go.Scatter(x=session_data['day_type'], y=session_data['pc_probe'], mode='lines', 
                                    line_color='darkgrey', showlegend=False, name=mouse, opacity=0.6), row=1, col=idx+1)
fig.update_yaxes(title='Probe Accuracy (%)', col=1)
fig.show()